# 1. Import Library

In [11]:
import os
import pandas as pd
import difflib
from collections import defaultdict
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


# 2. Load Processed Dataset

In [12]:
data_path = r"onehot_engineered_data_final.csv"


In [13]:
df = pd.read_csv(data_path)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 74 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Năm                                         315 non-null    int64  
 1   Tiền đầu tư giáo dục                        315 non-null    float64
 2   Dân số trung bình                           315 non-null    float64
 3   Mật độ dân số                               315 non-null    float64
 4   Điểm thi đại học trung bình của địa phương  315 non-null    float64
 5   Vùng kinh tế_0                              315 non-null    bool   
 6   Vùng kinh tế_1                              315 non-null    bool   
 7   Vùng kinh tế_2                              315 non-null    bool   
 8   Vùng kinh tế_3                              315 non-null    bool   
 9   Vùng kinh tế_4                              315 non-null    bool   
 10  Vùng kinh tế_5

In [15]:
df.describe()

,Năm,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
count,315.000000,315.000000,315.000000,315.000000,315.000000
mean,2020.000000,0.949023,7.135180,5.730927,5.939602
std,1.416464,0.848805,0.578922,0.993517,0.973514
min,2018.000000,0.027806,5.743003,3.912023,3.690686
25%,2019.000000,0.540165,6.774658,4.927254,5.238326
50%,2020.000000,0.750797,7.117287,5.605802,5.857310
75%,2021.000000,1.039475,7.430696,6.507278,6.589629
max,2022.000000,10.308810,9.147371,8.407602,8.283732


In [16]:
df.head()

,Năm,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương,Vùng kinh tế_0,Vùng kinh tế_1,Vùng kinh tế_2,Vùng kinh tế_3,Vùng kinh tế_4,...,Địa phương_53,Địa phương_54,Địa phương_55,Địa phương_56,Địa phương_57,Địa phương_58,Địa phương_59,Địa phương_60,Địa phương_61,Địa phương_62
0,2018,0.434603,6.741110,4.670490,4.382694,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2019,0.485911,6.754721,4.682131,4.215383,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2020,0.422106,6.768769,4.700480,5.221484,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2021,0.474379,6.787946,4.718499,5.680538,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2022,0.554896,6.794273,4.727388,5.299501,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
id_to_province = {
    0: 'An Giang',
    1: 'Bà Rịa - Vũng Tàu',
    2: 'Bình Dương',
    3: 'Bình Phước',
    4: 'Bình Thuận',
    5: 'Bình Định',
    6: 'Bạc Liêu',
    7: 'Bắc Giang',
    8: 'Bắc Kạn',
    9: 'Bắc Ninh',
    10: 'Bến Tre',
    11: 'Cao Bằng',
    12: 'Cà Mau',
    13: 'Gia Lai',
    14: 'Hà Giang',
    15: 'Hà Nam',
    16: 'Hà Nội',
    17: 'Hà Tĩnh',
    18: 'Hòa Bình',
    19: 'Hưng Yên',
    20: 'Hải Dương',
    21: 'Hải Phòng',
    22: 'Hậu Giang',
    23: 'Khánh Hòa',
    24: 'Kiên Giang',
    25: 'Kon Tum',
    26: 'Lai Châu',
    27: 'Long An',
    28: 'Lào Cai',
    29: 'Lâm Đồng',
    30: 'Lạng Sơn',
    31: 'Nam Định',
    32: 'Nghệ An',
    33: 'Ninh Bình',
    34: 'Ninh Thuận',
    35: 'Phú Thọ',
    36: 'Phú Yên',
    37: 'Quảng Bình',
    38: 'Quảng Nam',
    39: 'Quảng Ngãi',
    40: 'Quảng Ninh',
    41: 'Quảng Trị',
    42: 'Sóc Trăng',
    43: 'Sơn La',
    44: 'TP. Cần Thơ',
    45: 'TP. Hồ Chí Minh',
    46: 'Thanh Hóa',
    47: 'Thái Bình',
    48: 'Thái Nguyên',
    49: 'Thừa Thiên - Huế',
    50: 'Tiền Giang',
    51: 'Trà Vinh',
    52: 'Tuyên Quang',
    53: 'Tây Ninh',
    54: 'Vĩnh Long',
    55: 'Vĩnh Phúc',
    56: 'Yên Bái',
    57: 'Điện Biên',
    58: 'Đà Nẵng',
    59: 'Đắk Lắk',
    60: 'Đắk Nông',
    61: 'Đồng Nai',
    62: 'Đồng Tháp'
}


In [18]:
id_to_region = {
    0: 'Vùng Bắc Trung Bộ và duyên hải miền Trung',
    1: 'Vùng Tây Nguyên',
    2: 'Vùng trung du và miền núi phía Bắc',
    3: 'Vùng Đông Nam Bộ',
    4: 'Vùng đồng bằng sông Cửu Long',
    5: 'Vùng đồng bằng sông Hồng'
}

# 3. Define models

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

models = {
    "Linear Regression": {
        'model': LinearRegression(),
        'param_grid': {
            "fit_intercept": [True, False]
            }
    },
    "Polynomial Regression": {
        'model': Pipeline([
            ("poly", PolynomialFeatures()),
            ("linear", LinearRegression())
        ]),
        'param_grid': {
            "poly__degree": [2, 3],
            "linear__fit_intercept": [True, False]
        }
    },
    "Decision Tree": {
        'model': DecisionTreeRegressor(),
        'param_grid': {
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    "Random Forest": {
        'model': RandomForestRegressor(),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    "Gradient Boosting": {
        'model': GradientBoostingRegressor(),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    "Support Vector Regression": {
        'model': SVR(),
        'param_grid': {
            'kernel': ['linear', 'rbf'],
            'C': [0.1, 1, 10],
            'epsilon': [0.01, 0.1, 0.2]
        }
    },
    "LightGBM": {
        'model': LGBMRegressor(),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    "XGBoost": {
        'model': XGBRegressor(),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.8, 0.9, 1.0]
        }
    }
}

In [20]:
from sklearn.model_selection import GridSearchCV

results = {}

for model_name, model_info in models.items():
    print(f"Training {model_name}...")
    
    # Retrieve model and parameter grid
    model = model_info['model']
    param_grid = model_info['param_grid']
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(df.drop(columns=['Năm', 'Điểm thi đại học trung bình của địa phương']), df['Điểm thi đại học trung bình của địa phương'])
    
    # Collect results
    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    
    # Save results
    results[model_name] = {
        'best_model': best_model,
        'best_score': best_score,
        'best_param': grid_search.best_params_
    }

Training Linear Regression...
Training Polynomial Regression...
Training Decision Tree...
Training Random Forest...


In [ ]:
# Print results
for model_name, result in results.items():
    print(f"{model_name} - Best Score: {result['best_score']} - Best Params: {result['best_param']}")

Linear Regression - Best Score: -0.6670952687886248 - Best Params: {}
Polynomial Regression - Best Score: -0.6670952687886502 - Best Params: {'linear__fit_intercept': True, 'poly__degree': 1}
Decision Tree - Best Score: -0.7534036443082704 - Best Params: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5}
Random Forest - Best Score: -0.6429943592076531 - Best Params: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
Gradient Boosting - Best Score: -0.6369515949411995 - Best Params: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Support Vector Regression - Best Score: -0.6704135862148997 - Best Params: {'C': 0.1, 'epsilon': 0.2, 'kernel': 'linear'}
LightGBM - Best Score: -0.677324326212296 - Best Params: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
XGBoost - Best Score: -0.6448880242145133 - Best Params: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample'